In [16]:
!pip install transformers[sentencepiece]==4.28.0 datasets sacrebleu evaluate

In [19]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [21]:
import transformers

print(transformers.__version__)

4.29.2


In [44]:
from datasets import load_dataset

#wmt16_train = load_dataset("ethansimrm/wmt16_biomed", use_auth_token=True) #We won't need to use this for now

wmt16_test = load_dataset("ethansimrm/wmt16_biomed_test", use_auth_token=True)
wmt16_gold = load_dataset("ethansimrm/wmt16_biomed_gold", use_auth_token=True)

  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/ethansimrm--wmt16_biomed_test-195b3b7168424280/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/ethansimrm--wmt16_biomed_gold-4a81ce75a76006f6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

Let's aim low for now. Given a (small) pre-trained large language model - Marian or T5-Small, etc, we wish to evaluate baseline performance on the WMT16 Biomedical Test Set. 
- We have converted the test set and the gold-standard answers to CSV files and uploaded both to the HuggingFace Hub.
- Intuitively, we must extract source sentences from the test set and target sentences from the gold set, but only where there is a direct correspondence between them.
- We know for sure that "passage/0" (the scientific paper titles) - are one sentence long in both languages, so there is a direct correspondence - we cannot guarantee the same for the rest of the sentences.



In [ ]:
#The relevant columns for the abstract titles are:
source_sentences = wmt16_test["test"]["passage/0/sentence/0/text"]
target_sentences = wmt16_gold["train"]["passage/0/sentence/0/text"]

In [ ]:
from datasets import load_metric
metric = load_metric("sacrebleu")

In [89]:
#This used a 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Model, pipeline
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("ethansimrm/test_t5_small_example_kaggle3") 
translator = pipeline("translation", model=model, tokenizer=tokenizer, use_auth_token = True)
for s in source_sentences:
    source = "translate English to French: " + s
    if(source[len(source) - 1] == '.'):
        source = source[:-1]
    print(source)
    result = translator(source)
    print(result)

translate English to French: Response of broiler chicks to diets with bambaranut (Voandzeia subterranea) as a protein source
[{'translation_text': 'Réponse des bourreaux à la diète avec le bambaranois (Voandzeia subterranea) comme source de protéines'}]
translate English to French: Effects of reducing postpartum re-mating timeperiod on reproductive performance of the doe
[{'translation_text': ' Effects of reducing postpartum re-mating timeperiod on reproductive performance of the tee'}]
translate English to French: Factors influencing reproduction parameters of ewes Ouled Djellal
[{'translation_text': 'Factors influencing reproduction parameters of ewes Ouled Djellal'}]
translate English to French: Socioeconomic inequalities in child mortality: comparisons across nine developing countries
[{'translation_text': 'Socioeconomic inequalities in child mortality: comparisons across nine developing countries'}]
translate English to French: Defining and measuring health inequality: an approach

KeyboardInterrupt: 

Additional Information:
- We could simply truncate the gold-standard answers to the length of the source sentences, but we only require a quick-and-dirty evaluation today.
- Furthermore, the test set contains French abstracts which contain information which cannot be inferred from the English source; we cannot assume a 1-1 faithful translation.